In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login

login()

In [4]:
# Import libraries
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import pandas as pd

In [5]:
# Load the dataset
dataset_path = '/content/drive/MyDrive/RickAndMortyScripts.csv'
df = pd.read_csv(dataset_path)

In [ ]:
# Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

In [ ]:
# Fine-tuning parameters
batch_size = 4
epochs = 3
learning_rate = 2e-5

In [ ]:
# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(df) * epochs // batch_size
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
# DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {"text": f"{self.data['name'][idx]}: {self.data['line'][idx]}"}

custom_dataset = CustomDataset(df)
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs = tokenizer(batch["text"], return_tensors="pt", padding=True, truncation=True)
        inputs.to(device)
        labels = inputs["input_ids"].clone()
        labels[labels == tokenizer.pad_token_id] = -100
        labels.to(device)

        optimizer.zero_grad()

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{epochs} - Average Loss: {average_loss}")

In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_llama-2-7B_rick_and_morty")
tokenizer.save_pretrained("fine_tuned_llama-2-7B_rick_and_morty")